In [ ]:
import sys
import os
import glob
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy as np
import json

import cv2
# from google.colab.patches import cv2_imshow
# !pip3 install rich
from rich.progress import Progress

from utils import get_abs_path

In [ ]:
project_path = get_abs_path(1)

images_dir = project_path + 'data/maps/'
paths_dir = project_path + 'data/planned_maps/'
paths_filename = 'paths.json'

samples_count = 50
training_percentage = 0.9
test_percentage = 0.1

training_count = int(samples_count * training_percentage)
test_count = int(samples_count * test_percentage)

In [ ]:
paths = {}
with open(paths_dir + paths_filename) as f:
  paths = json.load(f)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.io import read_image
import torchvision.transforms as tt

In [ ]:
class ImageDataset(Dataset):

  def __init__(self, images_dir, transform = None, target_transform = None):

    self.images_dir = images_dir
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):

    files = []
    files.extend(glob.glob(self.images_dir + '*.png'))
    return len(files)

  def __getitem__(self, idx):

    image_path = self.images_dir + 'map_' + str(idx) + '.png'
    image = read_image(image_path)
    image = image[1:3, :, :] / 255
    # image = np.reshape(image, (1, 100, 100))

    state = []
    if self.transform:
        state = torch.get_rng_state()
        image_in = self.transform(image)
    if self.target_transform:
        torch.set_rng_state(state)
        image_out = self.target_transform(image)

    return image, image

In [ ]:
encoder_data = ImageDataset(images_dir)

training_encoder_data, test_encoder_data = torch.utils.data.random_split(encoder_data, (training_count, test_count))

train_encoder_dataloader = DataLoader(training_encoder_data, batch_size=16, shuffle=True, pin_memory=True)
test_encoder_dataloader = DataLoader(test_encoder_data, batch_size=16, shuffle=True, pin_memory=True)

In [ ]:
def display_image(image):
  image = image * 255
  image = image.squeeze()
  image = image.permute(1, 2, 0).detach().numpy()
  image = np.concatenate((image[:,:,1:2], image[:,:,0:1], (np.zeros((120, 120, 1)))), axis=2)
  cv2.imshow('', image)
  # cv2_imshow(image)

In [ ]:
train_features, train_labels = next(iter(train_encoder_dataloader))

display_image(train_features[0])

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {0} device'.format(device))

In [ ]:
def batch_norm_conv(ni, nf, stride=1):
  return nn.Sequential(
    nn.BatchNorm2d(ni),
    nn.ReLU(inplace=True),
    nn.Conv2d(ni, nf, (3, 3), stride=stride, padding=1, bias=False)
  )

In [ ]:
def conv1x1(ni, nf, stride=1):
  return nn.Sequential(
    nn.Conv2d(ni, nf, (1, 1), stride=stride),
    nn.BatchNorm2d(nf)
  )

In [ ]:
class ResBlock(nn.Module):

  def __init__(self, ni, nf, stride=1):
    super(ResBlock, self).__init__()
    self.shortcut = conv1x1(ni, nf, stride=stride)
    self.Bn1 = batch_norm_conv(ni, nf, stride=stride)
    self.Bn2 = batch_norm_conv(nf, nf)

  def forward(self, x):
    short = self.shortcut(x)
    x = self.Bn1(x)
    x = self.Bn2(x)
    return x.add(short)

In [ ]:
class ResEncoder(nn.Module):

  def __init__(self):
    super(ResEncoder, self).__init__()

    self.short1 = conv1x1(2, 2)
    self.conv1 = nn.Conv2d(2, 2, (3, 3), padding=1)
    self.Bn1 = batch_norm_conv(2, 2)
    self.Res1 = ResBlock(2, 4)
    self.Res2 = ResBlock(4, 8, stride=2)
    self.Res3 = ResBlock(8, 16, stride=2)
    self.Res4 = ResBlock(16, 16, stride=2)

  def forward(self, x):

    r = self.short1(x)
    x = self.conv1(x)
    x = self.Bn1(x)
    x = x.add(r)
    short1 = torch.clone(x)

    x = self.Res1(x)
    short2 = torch.clone(x)
    x = self.Res2(x)
    short3 = torch.clone(x)
    x = self.Res3(x)
    short4 = torch.clone(x)
    x = self.Res4(x)

    return x, short1, short2, short3, short4

In [ ]:
class ResDecoder(nn.Module):

  def __init__(self):
    super(ResDecoder, self).__init__()

    self.Up1 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.Up2 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.Up3 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.Up4 = nn.UpsamplingBilinear2d(scale_factor=1)

    self.Res5 = ResBlock(32, 8)
    self.Res6 = ResBlock(16, 4)
    self.Res7 = ResBlock(8, 2)
    self.Res8 = ResBlock(4, 2)

    self.conv2 = nn.Conv2d(2, 2, (1, 1))
    self.sigmoid = nn.Sigmoid()

  def forward(self, x, short1, short2, short3, short4):

    x = self.Up1(x)
    x = torch.cat((short4, x), dim=1)
    x = self.Res5(x)

    x = self.Up2(x)
    x = torch.cat((short3, x), dim=1)
    x = self.Res6(x)

    x = self.Up3(x)
    x = torch.cat((short2, x), dim=1)
    x = self.Res7(x)

    x = self.Up4(x)
    x = torch.cat((short1, x), dim=1)
    x = self.Res8(x)

    x = self.conv2(x)
    x = self.sigmoid(x)

    return x

In [ ]:
class ResUNet(nn.Module):

  def __init__(self):
    super(ResUNet, self).__init__()

    self.Encoder = ResEncoder()
    self.Decoder = ResDecoder()

  def forward(self, x):

    x, short1, short2, short3, short4 = self.Encoder(x)
    x = self.Decoder(x, short1, short2, short3, short4)
    return x

model = ResUNet().to(device)
print(model.Encoder)
print(model.Decoder)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):

  size = len(dataloader.dataset)
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    #print(pred.shape, y.shape)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')

In [ ]:
def test(dataloader, model, loss_fn):

  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  # correct /= size
  print(f'Test Error: \n Accuracy: {(np.nan):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [ ]:
epochs = 5
for t in range(epochs):
  print(f'Epoch {t+1}\n-------------------------------')
  train(train_encoder_dataloader, model, loss_fn, optimizer)
  test(test_encoder_dataloader, model, loss_fn)
print('Done!')

In [ ]:
test_features, test_labels = next(iter(test_encoder_dataloader))
display_image(test_features[0])
display_image(test_labels[0])

result = model(test_labels)
display_image(result[0])
print(model.Encoder(test_labels)[0][0].shape)